In [1]:
import os
from dotenv import load_dotenv
import wget

In [2]:
env = load_dotenv()
LLAMA_API_KEY = os.getenv('LLAMA_CLOUD_API_KEY')
OPENAI_KEY = os.getenv('OPEN_AI_API_KEY')
HF_KEY = os.getenv('HF_KEY')

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI

from llama_index.core import Settings



embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
llm = OpenAI(model="gpt-3.5-turbo", api_key= OPENAI_KEY)

Settings.embed_model = embed_model
Settings.llm = llm


In [4]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
# urls = [
#     "https://openreview.net/pdf?id=VtmBAGCN7o",
#     "https://openreview.net/pdf?id=6PmJoRfdaK",
#     "https://openreview.net/pdf?id=hSyW5go0v8",
# ]

# struc_doc_urls =[
#     "https://www.opm.gov/forms/pdfimage/sf50.pdf"
# ]

# files = []

# all_urls = urls + struc_doc_urls
# for url in all_urls:
#     file = wget.download(url=url)
#     files.append(file)


In [5]:
from PIL import Image
from llama_index.core import SimpleDirectoryReader

def parse_image(file_path):
    image = Image.open(file_path)
    text = LlamaParse.parse_image(image)
    return text

def base_reader(file_path):
    reader = LlamaParse(result_type = 'text')
    base_docs = reader.load_data(file_path)
    return base_docs

In [6]:
files = [
    "travel_doc_sample2.pdf",
    "travel_doc_sample.pdf",
]

In [139]:
# parser = LlamaParse(result_type='text')
# json_response = parser.get_json_result(files[0])
# page_lst = json_response[0]['pages']
# docs = []
# for item in page_lst:
#     docs.append(Document(
#             text=item.get("text"), 
#             metadata=item
#         )
#     )


'\n'

In [59]:
from llama_parse import LlamaParse
from llama_index.core.schema import Document

def _load(file_path: str) -> Document:
    parser = LlamaParse(result_type='text')
    json_response = parser.get_json_result(file_path)
    page_results = json_response[0]['pages']
    docs = []
    for item in page_results:
        doc = Document(
                text=item.get("text"), 
                metadata={"page_content":item["page"]}
            )
    docs.append(doc)
    return docs

In [ ]:
docs = _load(file_path)
    # node_parser = LlamaParseJsonNodeParser(llm = llm, num_workers=16, include_metadata=True)
    # nodes = node_parser.get_nodes_from_documents(docs)
    # base_nodes, objects = node_parser.get_nodes_and_objects(nodes)
    # vector_index = VectorStoreIndex(nodes = base_nodes + objects)
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(docs)
vector_index = VectorStoreIndex(nodes = nodes)

def use_query_engine(query:str, )->str:
    query_engine = vector_index.as_query_engine(similarity_top_k=5, verbose=True)
    res = query_engine.query(query)
    return res
    
    name = Path(file_path).stem.split(" ")[0]

In [61]:
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.core.node_parser import SentenceSplitter, LlamaParseJsonNodeParser
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters, FilterCondition
from pathlib import Path

def get_vec_tools(file_path:str) -> str:
    docs = _load(file_path)
    node_parser = LlamaParseJsonNodeParser(llm = llm, num_workers=16, include_metadata=True)
    nodes = node_parser.get_nodes_from_documents(docs)
    base_nodes, objects = node_parser.get_nodes_and_objects(nodes)
    recursive_index = VectorStoreIndex(nodes = base_nodes + objects)
    # splitter = SentenceSplitter(chunk_size=1024)
    # nodes = splitter.get_nodes_from_documents(docs)
    # vector_index = VectorStoreIndex(nodes = nodes)

    def use_query_engine(query:str)->str:
        query_engine = recursive_index.as_query_engine(similarity_top_k=5, verbose=True)
        res = query_engine.query(query)
        return res
    
    name = Path(file_path).stem.split(" ")[0]
    query_tool = FunctionTool.from_defaults(
        name=f"vector_tool_{name}",
        fn=use_query_engine
    )
    return query_tool
        

In [62]:
paper_to_tools_dict = {}
for file in files:
    file_query_tool = get_vec_tools(file)
    paper_to_tools_dict[file] = file_query_tool

Started parsing the file under job_id cac11eca-4608-4f72-bddb-c4048e9d3a24
Started parsing the file under job_id cac11eca-8b2a-42db-9f71-1df6d4a68cda


In [22]:
from llama_index.core.agent import ReActAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = ReActAgentWorker.from_tools(
    [tool for tool in paper_to_tools_dict.values()],
    # llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [23]:
response = agent.query(
    "Across all documents, can you provide the dates of travel?"
)

print(str(response))

AttributeError: 'list' object has no attribute 'metadata'